In [ ]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set_style('whitegrid')

In [ ]:
delinq = pd.read_csv('cs-training.csv', index_col = 'Unnamed: 0')
delinq.head(2)

In [ ]:
sns.heatmap(delinq.isnull())

In [ ]:
from sklearn.model_selection import train_test_split
train, test= train_test_split(delinq, test_size=0.18, random_state=42)
train, valid = train_test_split(train, test_size=0.20, random_state=42)  #splitted data into train, validation, test set

In [ ]:
train.shape, valid.shape, test.shape

In [ ]:
train.info()

In [ ]:
train.describe()

In [ ]:
#sns.pairplot(train)

In [ ]:
sns.boxplot(train['RevolvingUtilizationOfUnsecuredLines'])  #data contains outliers,
                                                            #the 'RevolvingUtilizationOfUnsecuredLines for example

In [ ]:
valid.describe()

In [ ]:
train['MonthlyIncome'].quantile(0.99)

In [ ]:
train = train.drop(train[train['MonthlyIncome'] > 25000].index)   #dropped rows with
train.shape                                                       #value greater than the 99th percentil

In [ ]:
train['NumberRealEstateLoansOrLines'].quantile(0.99)

In [ ]:
train = train.drop(train[train['NumberRealEstateLoansOrLines'] > 3].index)   #dropped rows with
train.shape                                                                  #value greater than the 99th percentile

In [ ]:
train['NumberOfDependents'].quantile(0.99)

In [ ]:
train = train.drop(train[train['NumberOfDependents'] > 4].index)
train.shape

In [ ]:
train.shape

In [ ]:
def getoutliers(a, df):                                         #returns rows outside the lower and upper limit, 
    q1 = a.quantile(0.25) - a.min()                             #that is, the suspected outliers             
    q3 = a.quantile(0.75) - a.quantile(0.50) 
    iqr = a.quantile(0.75) - a.quantile(0.25) 
    lower_limit = q1 - (1.5 *iqr)
    upper_limit = q3 + (1.5*iqr)
    a = df[(a < lower_limit) | (a > upper_limit)]
    return a

In [ ]:
outlier_df = getoutliers(train['RevolvingUtilizationOfUnsecuredLines'], train)  #dropped the rows with 
train = train.drop(outlier_df.index)                                            #the outliers
train.shape

In [ ]:
train['NumberOfDependents'] = train['NumberOfDependents'].astype('Int64') #converted float to integer

In [ ]:
train.describe()

In [ ]:
train.head(2)

In [ ]:
train = train.drop(train[train['MonthlyIncome'].isnull()].index)  #dropped MonthlyIncome Nulls  

valid = valid.drop(valid[valid['MonthlyIncome'].isnull()].index)

test = test.drop(test[test['MonthlyIncome'].isnull()].index)

In [ ]:
train.shape

In [ ]:
sns.heatmap(train.isnull())

In [ ]:
train = train.drop(train[train['NumberOfDependents'].isnull()].index)  #dropped NumberOfDependents Nulls

valid = valid.drop(train[train['NumberOfDependents'].isnull()].index)

test = test.drop(test[test['NumberOfDependents'].isnull()].index)

In [ ]:
sns.heatmap(train.isnull())

In [ ]:
train_y = train['SeriousDlqin2yrs']
train_X = train.drop('SeriousDlqin2yrs', axis = 1)

valid_y = valid['SeriousDlqin2yrs']
valid_X = valid.drop('SeriousDlqin2yrs', axis = 1)

test_y = test['SeriousDlqin2yrs']
test_X = test.drop('SeriousDlqin2yrs', axis = 1)


In [ ]:
from imblearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA
from xgboost import XGBClassifier
from sklearn.preprocessing import PolynomialFeatures
from sklearn.impute import SimpleImputer

In [ ]:
pipeline = make_pipeline(PolynomialFeatures(degree = 4),
                         StandardScaler(),
                         PCA(n_components = 5),
                         SMOTE(random_state = 42),
                         XGBClassifier(n_jobs = -1, n_estimators = 200, learning_rate = 0.1, colsample_bylevel =0.8,
                         colsample_bytree = 0.8))

In [ ]:
from sklearn.model_selection import cross_val_score, KFold

In [ ]:
folds = KFold(n_splits=5, shuffle= True, random_state= 0)

In [ ]:
#cross_val_score(pipeline, train_X, train_y, scoring= 'recall',cv = folds,verbose= True, n_jobs= -1).mean()

In [ ]:
pipeline.fit(train_X, train_y)

In [ ]:
prediction = pipeline.predict(valid_X)
#prediction = pipeline.predict(test_X)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
print(classification_report(valid_y,prediction))
#print(classification_report(test_y,prediction))

In [ ]:
print(confusion_matrix(valid_y,prediction))
#print(confusion_matrix(test_y,prediction))